In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import jieba
from jieba import lcut, analyse, Tokenizer
from wordcloud import WordCloud, STOPWORDS
from gensim.models import word2vec
from gensim.models.keyedvectors import KeyedVectors

In [2]:
def Openfile(path):
    f = open(path, "r", encoding="utf-8")
    result = f.read()
    f.close()
    return result
def Openfile_sp(path):
    f = open(path, "r", encoding="utf-8")
    result = f.readlines()
    f.close()
    return result

In [54]:
# open file
stopwords_file = 'datafile/stop_words.txt'
jieba.set_dictionary('./dict.txt.big.txt')
jieba.load_userdict('datafile/user_dict.txt')
jieba.analyse.set_stop_words('datafile/stop_words.txt')

evaair_file = 'datafile/EvaAir_text.csv'
df = pd.read_csv(evaair_file)

Building prefix dict from D:\WORK5\TechProject\TechProject2019\dict.txt.big.txt ...
Loading model from cache C:\Users\ASUS\AppData\Local\Temp\jieba.u15c153cb84f92c395787873e58f7016c.cache
Loading model cost 0.984 seconds.
Prefix dict has been built succesfully.


In [55]:
dec = Openfile_sp(stopwords_file)
stopwords = set()
for i in range(len(dec)):
    stopwords.add(dec[i].strip())

In [56]:
df1 = df['text'].tolist()
for i in range(len(df1)):
    if isinstance(df1[i], float):
        df1[i] = str(df1[i])
evaair_text = []
for i in range(len(df1)):
    evaair_text.append(df1[i].strip())

In [57]:
result = []
for i in range(len(evaair_text)):
    r = lcut(evaair_text[i], cut_all=False, HMM=False)
    result.append(r)
    

adj_result = []
for i in range(len(result)):
    par = []
    for j in range(len(result[i])):
        if len(result[i][j]) > 1:
            if result[i][j] not in stopwords:
                par.append(result[i][j])
    adj_result.append(par)

In [77]:
E_model = word2vec.Word2Vec(adj_result, size=200, min_count=10)
E_model.save("model/EvaAir_model.bin")

In [101]:
model = word2vec.Word2Vec.load("model/EvaAir_model.bin")
word_vectors = model.wv

In [102]:
res = word_vectors.wv.most_similar('長榮航空', topn = 20)

D:\Program Files\Anaconda\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """Entry point for launching an IPython kernel.


In [103]:
res

[('一同', 0.9990308284759521),
 ('美好', 0.9988092184066772),
 ('期許', 0.9987189173698425),
 ('安全', 0.9986463785171509),
 ('飛行', 0.9986349940299988),
 ('體驗', 0.9986276626586914),
 ('未來', 0.9985326528549194),
 ('邀請', 0.9984458684921265),
 ('旅程', 0.998437762260437),
 ('今天', 0.9982931613922119),
 ('旅行', 0.9982914924621582),
 ('影片', 0.9982848763465881),
 ('設計', 0.9982648491859436),
 ('姐妹', 0.9981586933135986),
 ('舒適', 0.9981505870819092),
 ('加入', 0.9981184005737305),
 ('各位', 0.9981131553649902),
 ('世界', 0.9979893565177917),
 ('機上', 0.9979846477508545),
 ('希望', 0.997948169708252)]

In [88]:
# test part
key_word = 'datafile/evaair_keywords.csv'
df = pd.read_csv(key_word)
key = set()
df1 = df['key'].tolist()
for i in range(len(df1)):
    key.add(df1[i])
    
key_result = []
for i in range(len(adj_result)):
    par = []
    for j in range(len(adj_result[i])):
        if adj_result[i][j] in key:
            par.append(adj_result[i][j])
    key_result.append(par)
key_result

[['長榮航空', '小潔', '即日起', '留言', '幸運', '機會', '長榮航空', '長榮航空', '長榮航空', '長榮航空'],
 ['日本', '旅遊', '日本', '美麗', '小潔', '行程', '搭乘', '日本', '旅程', 'BR', 'TWD', '未稅'],
 ['期間',
  '旅遊',
  '期間',
  '出國',
  '行程',
  '期間',
  '旅客',
  '長榮',
  '立榮航空',
  '桃園',
  '機場',
  '出發',
  '航班',
  '即日起',
  '期間',
  '服務',
  '報到',
  '時間',
  '報到',
  '時間',
  '航班',
  '航班',
  '報到',
  '前往',
  '登機',
  '登機',
  '航班',
  '航班',
  '長榮航空',
  '手機',
  'APP',
  '長榮航空',
  '網站',
  '報到',
  '時間',
  '長榮航空',
  '報到',
  '手機',
  'APP',
  '報到',
  '報到'],
 ['長榮', '小潔'],
 [],
 ['出國', '小潔', '行程', '企鵝', 'BR', '飛航', '期間', 'TWD', '未稅'],
 ['夢想', '長榮', '長榮航空', '長榮', '服務', '2018', '服務', '夢幻客機'],
 ['小潔', '小潔', '長榮', '設計', '舒適', '設計', '長榮航空', '機上', '時間', '粉絲團', '訊息'],
 ['長榮', '長榮', '旅行', '長榮航空', 'EVAAIR'],
 ['長榮',
  '航空',
  '即日起',
  '透過',
  '長榮航空',
  'EVA',
  'APP',
  '出發',
  '長榮',
  '機票',
  '開始',
  '旅行',
  '出發',
  '小潔',
  '提供',
  '服務',
  '長榮航空',
  '購票'],
 ['長榮航空', '桃園', '航班', '前往', '詳細', '資訊'],
 ['長榮航空',
  '全球',
  '安全',
  '航空公司',
  '小潔',
  '分享',
  '國際',
  '航空',
  '

In [89]:
E_test_model = word2vec.Word2Vec(key_result, size=300, min_count=10)
E_model.save("model/Evaair_key_model.bin")

In [104]:
key_model = word2vec.Word2Vec.load("model/Evaair_key_model.bin")
word_vectors = key_model.wv

In [105]:
res = word_vectors.wv.most_similar('長榮航空', topn = 20)

D:\Program Files\Anaconda\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """Entry point for launching an IPython kernel.


In [106]:
res

[('一同', 0.9990308284759521),
 ('美好', 0.9988092184066772),
 ('期許', 0.9987189173698425),
 ('安全', 0.9986463785171509),
 ('飛行', 0.9986349940299988),
 ('體驗', 0.9986276626586914),
 ('未來', 0.9985326528549194),
 ('邀請', 0.9984458684921265),
 ('旅程', 0.998437762260437),
 ('今天', 0.9982931613922119),
 ('旅行', 0.9982914924621582),
 ('影片', 0.9982848763465881),
 ('設計', 0.9982648491859436),
 ('姐妹', 0.9981586933135986),
 ('舒適', 0.9981505870819092),
 ('加入', 0.9981184005737305),
 ('各位', 0.9981131553649902),
 ('世界', 0.9979893565177917),
 ('機上', 0.9979846477508545),
 ('希望', 0.997948169708252)]